In [5]:
# Solo necesario en caso de problemas con los certificados SSL
import os
import certifi
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()

In [1]:
from transformers import BertForMaskedLM, pipeline

# Agregamos una prueba para verificar si estamos usando cuda o cpu
# e imprimimos el dispositivo que se está utilizando así como su nombre

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Dispositivo utilizado:", device)
if device == "cuda":
    print("Nombre del dispositivo:", torch.cuda.get_device_name(0))

Dispositivo utilizado: cpu


In [7]:
bert_lm = BertForMaskedLM.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
bert_lm     # Inspeccionamos el modelo cargado

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [9]:
# Los pipelines nos ayudan a definir la manera en que alimentaremos al modelo para el caso que estamos trabajando
bert_fill = pipeline('fill-mask', model=bert_lm, tokenizer='bert-base-cased')

Device set to use cuda:0


In [10]:
print(type(bert_fill.model))  # Verificamos que tipo de pipeline y modelo tenemos

preds = bert_fill(f"If you have a pet you will most likely spend a lot in {bert_fill.tokenizer.mask_token} and medicines.")

# [MASK]

print("Frase: If you have a pet you will most likely spend a lot in ***** and medicines.")
print("Probables frases completas:")

for pred in preds:
    print(f"Predicción: {pred['sequence']}")
    print(f"Score: {pred['score']:.4f}")


<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>
Frase: If you have a pet you will most likely spend a lot in ***** and medicines.
Probables frases completas:
Predicción: If you have a pet you will most likely spend a lot in food and medicines.
Score: 0.2605
Predicción: If you have a pet you will most likely spend a lot in money and medicines.
Score: 0.0651
Predicción: If you have a pet you will most likely spend a lot in medicine and medicines.
Score: 0.0351
Predicción: If you have a pet you will most likely spend a lot in drugs and medicines.
Score: 0.0192
Predicción: If you have a pet you will most likely spend a lot in books and medicines.
Score: 0.0179


d:\git\fundamentos-llms\fund-llms\Lib\site-packages\transformers\models\bert\modeling_bert.py:412: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
